## Part 1: External libraries

In [1]:
# Set up for Colab: tf2.1 and PyDrive to download the dataset
# installing tensorflow 2.1
!pip install --quiet tensorflow-gpu==2.12.0
!pip install tensorflow==2.12.0

# After this, restart the runtime to avoid authentication issues
!pip install -q -U PyDrive
!pip install tensorflow-gpu==2.12.0


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: 

In [2]:
import os                       # pkg to read filepaths from the dataset folder
import tensorflow as tf         # tensorflow
import tensorflow.keras as K    # keras API
import numpy as np              # numpy to work with tensors (with tf)
import matplotlib.pyplot as plt # function to show images
%matplotlib inline
import cv2                      # pkg to load image as np.array

# vgg net for perceptual loss
from tensorflow.keras.applications.vgg19 import VGG19

# Import PyDrive2 and associated libraries.
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Part 2: Mounting Google Drive and Importing External Libraries

In [3]:
# Authenticate and create the PyDrive2 client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#avishai adding
# Mount Google Drive if using Colab
try:
    from google.colab import drive as colab_drive
    colab_drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: Using Google Colab")
    %tensorflow_version 2.x  # Assuming you want to switch to TensorFlow 2.x
except:
    print("Note: Not using Google Colab")
    COLAB = False



Mounted at /content/drive
Note: Using Google Colab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
# Specifying the file ID
file_id = '19Q0BLy0jyxjZsa-4V_oM9Cw2QTIIlGeZ'
downloaded = drive.CreateFile({'id': file_id})

# Download the file to the Colab environment
downloaded.GetContentFile('Linnaeus_5_bigger.zip')

# Check the files in the current directory to confirm the zip file is downloaded
!ls

# Unzip the downloaded file
!unzip -q Linnaeus_5_bigger.zip

# List contents of the directory to confirm the extraction
!ls Linnaeus_5_bigger

drive  Linnaeus_5_bigger.zip  sample_data
test_128  train_256


## Part 3: Code


In [5]:
# defining the hyperparamters
input_size_test = 128 # 128x128x3 images (training_set)
input_size_train = 256 # 128x128x3 images (training_set)
batch_size= 16
num_epochs=20
noise_param=0.1

In [6]:
# Perceptual loss function

# load pre-trained (imagenet) vgg network, excluding fully-connected layer on the top
vgg = VGG19(include_top=False, weights='imagenet', input_shape=(None,None,3))
vgg_layer = K.Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
# make the net not trainable
for l in vgg_layer.layers: l.trainable=False

print(vgg_layer.summary())

def perceptual_loss(y_true,y_pred):
    '''This function computes the perceptual loss using an already trained VGG layer'''
    y_t=vgg_layer(y_true)
    y_p=vgg_layer(y_pred)
    loss=K.losses.mean_squared_error(y_t,y_p)
    return loss

80134624/80134624 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                              

In [7]:
# defining other metrics:
def psnr(y_true,y_pred):
    return tf.image.psnr(y_true,y_pred,1.0)
def ssim(y_true,y_pred):
    return tf.image.ssim(y_true,y_pred,1.0)

In [8]:
def image_proc_net(training_mode=True):
    """
    Function which creates the model to preprocess images.
    2 different types of model with different layers depending
    on whether it is in training mode or in testing mode
    """
    inputs = K.layers.Input((None, None, 3))
    if training_mode:
        # training mode: downsampling, gaussian noise and upsampling
        x = K.layers.MaxPool2D(pool_size=(2,2))(inputs)
        x = K.layers.GaussianNoise(noise_param)(x)  # Adjusted stddev to be within the valid range
        x = K.layers.UpSampling2D((2,2))(x)
    else:
        # testing mode: just an upsampling
        x = K.layers.UpSampling2D((2,2))(inputs)

    model = K.models.Model(inputs, x)

    for l in model.layers: l.trainable=False

    return model

# Instantiating the two models (for train and test)
image_proc_train = image_proc_net(training_mode=True)
image_proc_test = image_proc_net(training_mode=False)


In [9]:
#training_mode
print(image_proc_train.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, None, None, 3)    0         
 )                                                               
                                                                 
 gaussian_noise (GaussianNoi  (None, None, None, 3)    0         
 se)                                                             
                                                                 
 up_sampling2d (UpSampling2D  (None, None, None, 3)    0         
 )                                                               
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

In [10]:
#testing_mode
print(image_proc_test.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 up_sampling2d_1 (UpSampling  (None, None, None, 3)    0         
 2D)                                                             
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
def image_preprocess(image, training_mode=True):
    """
    Function which preprocess automatically an image
    wether it's in training or testing mode

    input:
      - image: np array (image tensor)
      - training_mode: binary flag
    """
    image = np.expand_dims(image,axis=0)
    if training_mode:
        return tf.squeeze(image_proc_train(image))
    else:
        return tf.squeeze(image_proc_test(image))

In [12]:
# reading image
def read_image(img_path):
    """
    function which loads an image from a path and convert it into np array
    """
    img = cv2.imread(img_path)
    b,g,r = cv2.split(img)   # cv2 reads BGR instead of canonical RGB
    img = cv2.merge([r,g,b]) # Switching it to RGB

    return img


In [13]:
import os
import numpy as np

all_images = []  # All train image paths

# 256x256 in training phase
path = 'Linnaeus_5_bigger/train_256'

# Check if the path exists
if not os.path.exists(path):
    print(f"Path does not exist: {path}")
else:
    files = os.listdir(path)
    for file in files:
        path_file = os.path.join(path, file)
        for dirpath, dirnames, filenames in os.walk(path_file):
            # Append full path to filenames
            filenames = [os.path.join(dirpath, x) for x in filenames]
            all_images.extend(filenames)

    # Picking only 2000 images
    all_images = all_images[:2000]
    np.random.shuffle(all_images)  # Random shuffling

    # Keep 85% for train and 15% for validation
    train_len = int(len(all_images) * 0.85)

    train_list = all_images[:train_len]
    val_list = all_images[train_len:]

    print(f"Total images: {len(all_images)}")
    print(f"Training images: {len(train_list)}")
    print(f"Validation images: {len(val_list)}")


# test
# for the test set we opted to take the same dataset
# but with images of size 128x128 instead of 256x256

test_list = []

path = 'Linnaeus_5_bigger/test_128'
files = os.listdir(path)
for file in files:
    path_file = path+'/'+file
    for (dirpath, dirnames, filenames) in os.walk(path_file):
        filenames = list(map(lambda x:path_file+'/'+x,filenames))
        test_list.extend(filenames)

np.random.shuffle(test_list) # random shuffling

test_list = test_list[:500]

print('images: ')
print('train:', len(train_list), 'val:', len(val_list), 'test:', len(test_list))

Total images: 2000
Training images: 1700
Validation images: 300
images: 
train: 1700 val: 300 test: 500


In [14]:
# defining train, validation and test generator

def train_generator():
    global batch_size
    while True:
        for start in range(0, len(train_list), batch_size):
                    x_batch = []
                    y_batch = []
                    end = min(start + batch_size, len(train_list))
                    ids_train_batch = train_list[start:end]
                    for i,ids in enumerate(ids_train_batch):
                        img_y = read_image(ids)
                        img_x = image_preprocess(img_y, training_mode=True)
                        x_batch.append(np.array(img_x,np.float32)/255.)
                        y_batch.append(np.array(img_y,np.float32)/255.)
                    x_batch = np.array(x_batch)
                    y_batch = np.array(y_batch)
                    yield x_batch,y_batch

def valid_generator():
    global batch_size
    while True:
        for start in range(0, len(val_list), batch_size):
                    x_batch = []
                    y_batch = []
                    end = min(start + batch_size, len(val_list))
                    ids_val_batch = val_list[start:end]
                    for i,ids in enumerate(ids_val_batch):
                        img_y = read_image(ids)
                        img_x = image_preprocess(img_y, training_mode=True)
                        x_batch.append(np.array(img_x,np.float32)/255.)
                        y_batch.append(np.array(img_y,np.float32)/255.)
                    x_batch = np.array(x_batch)
                    y_batch = np.array(y_batch)
                    yield x_batch,y_batch


In [15]:
# pixel shuffle
def pixel_shuffle(scale):
    '''
    This function implements pixel shuffling.
    ATTENTION: the scale should be bigger than 2, otherwise just returns the input.
    '''
    if scale > 1:
        return lambda x: tf.nn.depth_to_space(x, scale)
    else:
        return lambda x:x

In [16]:
#blocks definition for upscaling/downscaling

def add_down_block(x_inp, filters, kernel_size=(3, 3), padding="same", strides=1,r=False):
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = K.layers.BatchNormalization()(x)
    x = K.layers.Activation('relu')(x)
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = K.layers.BatchNormalization()(x)
    if r:
        # if r=True then we import an (1X1) Conv2D after input layer
        # in order the dimensions of 2 tensors coincide.
        x_inp = K.layers.Conv2D(filters,(1,1), padding=padding, strides=strides)(x_inp)
    x = K.layers.Add()([x,x_inp])
    return x

def add_up_block(x_inp, skip, filters, kernel_size=(3, 3), padding="same", strides=1, upscale_factor=2):
    # Calculate the upsampling factor based on the difference in input sizes
    upsampling_factor = skip.shape[1] // x_inp.shape[1]

    x = K.layers.UpSampling2D(size=(upsampling_factor, upsampling_factor))(x_inp)

    # Adjust the shape of the skip connection tensor
    skip = K.layers.Conv2D(filters, kernel_size=(1, 1), padding="same")(skip)

    x = K.layers.Concatenate()([x, skip])
    x = K.layers.BatchNormalization()(x)
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = K.layers.Activation('relu')(x)
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = K.layers.Activation('relu')(x)
    return x

def add_bottleneck(x_inp,filters, kernel_size=(3, 3), padding="same", strides=1):
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = K.layers.Activation('relu')(x)
    return x

In [17]:
#RUnet architecture layers
def RUNet(input_size_train=256, input_size_test=128):
    """
    Implementing with Keras the Robust UNet Architecture as proposed by
    Xiaodan Hu, Mohamed A. Naiel, Alexander Wong, Mark Lamm, Paul Fieguth
    in "RUNet: A Robust UNet Architecture for Image Super-Resolution"
    """
    with tf.device('/device:GPU:0'):  # This line ensures the model is built on the GPU
        inputs = K.layers.Input((input_size_train, input_size_train, 3))

        down_1 = K.layers.Conv2D(64, (7, 7), padding="same", strides=1)(inputs)
        down_1 = K.layers.BatchNormalization()(down_1)
        down_1 = K.layers.Activation('relu')(down_1)

        down_2 = K.layers.MaxPool2D(pool_size=(2, 2))(down_1)
        down_2 = add_down_block(down_2, 64)
        down_2 = add_down_block(down_2, 64)
        down_2 = add_down_block(down_2, 64)
        down_2 = add_down_block(down_2, 128, r=True)

        down_3 = K.layers.MaxPool2D(pool_size=(2, 2), strides=2)(down_2)
        down_3 = add_down_block(down_3, 128)
        down_3 = add_down_block(down_3, 128)
        down_3 = add_down_block(down_3, 128)
        down_3 = add_down_block(down_3, 256, r=True)

        down_4 = K.layers.MaxPool2D(pool_size=(2, 2))(down_3)
        down_4 = add_down_block(down_4, 256)
        down_4 = add_down_block(down_4, 256)
        down_4 = add_down_block(down_4, 256)
        down_4 = add_down_block(down_4, 256)
        down_4 = add_down_block(down_4, 256)
        down_4 = add_down_block(down_4, 512, r=True)

        down_5 = K.layers.MaxPool2D(pool_size=(2, 2))(down_4)
        down_5 = add_down_block(down_5, 512)
        down_5 = add_down_block(down_5, 512)
        down_5 = K.layers.BatchNormalization()(down_5)
        down_5 = K.layers.Activation('relu')(down_5)

        bn_1 = add_bottleneck(down_5, 1024)
        bn_2 = add_bottleneck(bn_1, 512)

        up_1 = add_up_block(bn_2, down_5, 512, upscale_factor=input_size_train // input_size_test)
        up_2 = add_up_block(up_1, down_4, 384, upscale_factor=2)
        up_3 = add_up_block(up_2, down_3, 256, upscale_factor=2)
        up_4 = add_up_block(up_3, down_2, 96, upscale_factor=2)

        up_5 = pixel_shuffle(scale=2)(up_4)
        up_5 = K.layers.Concatenate()([up_5, down_1])
        up_5 = K.layers.Conv2D(99, (3, 3), padding="same", strides=1)(up_5)
        up_5 = K.layers.Activation('relu')(up_5)
        up_5 = K.layers.Conv2D(99, (3, 3), padding="same", strides=1)(up_5)
        up_5 = K.layers.Activation('relu')(up_5)

        outputs = K.layers.Conv2D(3, (1, 1), padding="same")(up_5)
        model = K.models.Model(inputs, outputs)
        return model

model = RUNet()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  9472        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                           

In [ ]:
opt=K.optimizers.Adam(learning_rate=0.001) # Adam optimizer
model.compile(optimizer=opt,loss=perceptual_loss,metrics=[psnr,ssim,K.losses.mean_squared_error])
history = model.fit_generator(generator=train_generator(),
                              steps_per_epoch=np.ceil(float(len(train_list)) / float(batch_size)),
                              epochs=num_epochs,
                              verbose=1,
                              validation_data=valid_generator(),
                              shuffle=True,
                              validation_steps=np.ceil(float(len(val_list)) / float(batch_size)))


<ipython-input-18-221c1fa71950>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator(),


Epoch 1/20
107/107 [==============================] - 4515s 42s/step - loss: 7.1935 - psnr: 11.9597 - ssim: 0.1610 - mean_squared_error: 0.0956 - val_loss: 17.4798 - val_psnr: 11.2495 - val_ssim: 0.1924 - val_mean_squared_error: 0.0843
Epoch 2/20
 62/107 [================>.............] - ETA: 31:13 - loss: 3.4699 - psnr: 17.1211 - ssim: 0.3087 - mean_squared_error: 0.0206

## Part 4 - Experimental evaluation

#### Metrics: history

We show in the following part the results in term of metrics during the training epochs.

In [ ]:
def history_results(history, par1='loss', par2='val_loss', title='loss'):
    """
    Plot the history of the the 2 metrics (par1, par2) during
    the training (epochs)
    """
    plt.plot(history.history[par1])
    plt.plot(history.history[par2])
    plt.title(title)
    plt.ylabel(par1)
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.grid()
    plt.show()

    return
# print(history)
import os

path = '/content/drive/MyDrive/ProjectPhaseB/super_resolution_weights.h5'

# Ensure the directory exists, create it if necessary
directory = os.path.dirname(path)
if not os.path.exists(directory):
    os.makedirs(directory)


# # Assuming 'model' is your trained model
model.save_weights(path)
print(f"Model weights saved to '{path}'.")

In [ ]:
import os
print(os.listdir(os.getcwd()))


In [ ]:
history_results(history)

In [ ]:
history_results(history, 'psnr', 'val_psnr', 'psnr')

In [ ]:
history_results(history, 'ssim', 'val_ssim', 'ssim')

In [ ]:
history_results(history, 'mean_squared_error', 'val_mean_squared_error', 'means_squared_error')

#### Showing some visual results of Super-Resolution

In testing mode, 128x128 pixels images are used and then the result it's compared with 256x256 pixels images.

In [ ]:
def show_pictures(img_idx, x_batch, y_batch):
    """
    Function which shows 3 images:
    - Ground truth: High Resolution image
    - Low Resolution image
    - Super Resolution image using our trained model
    """
    fig = plt.figure(figsize=(15,18))

    ax1 = fig.add_subplot(1,3,1)
    im = model(np.expand_dims(x_batch[img_idx],axis=0))
    im = np.squeeze(im)
    #ax1.imshow((abs(im) * 255).astype(np.uint8))
    ax1.imshow(abs(im))
    ax1.set_title('Super Resolution (from LR)')

    ax2 = fig.add_subplot(1,3,2)
    #ax2.imshow(x_batch[img_idx] * 255).astype(np.uint8))
    ax2.imshow(x_batch[img_idx])
    ax2.set_title('Low Resolution')

    ax3 = fig.add_subplot(1,3,3)
    #ax3.imshow((y_batch[img_idx] * 255).astype(np.uint8))
    ax3.imshow(y_batch[img_idx])
    ax3.set_title('Ground truth')

    return

In [ ]:
# trying to visualize the results with some training images

chunk_size=20

x_chunk = []
y_chunk = []

ids_train_chunk = train_list[0:chunk_size]
for i,ids in enumerate(ids_train_chunk):
    img_y = read_image(ids)
    img_x = image_preprocess(img_y)
    x_chunk.append(np.array(img_x,np.float32)/255.)
    y_chunk.append(np.array(img_y,np.float32)/255.)
x_chunk = np.array(x_chunk)
y_chunk = np.array(y_chunk)

In [ ]:
x_chunk.shape

In [ ]:
show_pictures(7, x_chunk, y_chunk)
show_pictures(16, x_chunk, y_chunk)
show_pictures(19, x_chunk, y_chunk)

In [ ]:
import os
import cv2
import numpy as np

def read_image(img_path):
    """
    Reads an image from a file path.
    Converts from BGR to RGB format.
    """
    img = cv2.imread(img_path)
    if img is None:
        print(f"Error: Unable to read image at {img_path}")
        return None
    b, g, r = cv2.split(img)   # cv2 reads BGR instead of canonical RGB
    img = cv2.merge([r, g, b]) # Switching it to RGB
    return img

def get_image_paths(root_dir, size):
    """
    Collects image paths from the directory and filters by size.
    """
    image_paths = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if size in filename:  # Assuming size (e.g., '64' or '256') is in the filename
                image_paths.append(os.path.join(dirpath, filename))
    return image_paths

# Assuming image_preprocess is defined elsewhere

# Get all 256x256 image paths for training
train_root_dir = 'Linnaeus_5_bigger/train_256'
train_size = '256'
train_list = get_image_paths(train_root_dir, train_size)

# Get all 128x128 image paths for testing
test_root_dir = 'Linnaeus_5_bigger/test_128'
test_size = '128'
test_list = get_image_paths(test_root_dir, test_size)

# Prepare training data
x_train_chunk = []
y_train_chunk = []
train_chunk_size = 10  # Adjust the chunk size as needed
ids_train_chunk = train_list[:train_chunk_size]

for i, ids in enumerate(ids_train_chunk):
    # Construct the path for the corresponding 256x256 image
    ids_256 = ids.replace(train_size, train_size)
    print(f"Processing: 256x256 image at {ids_256}")

    # Check if the 256x256 image file exists
    if not os.path.isfile(ids_256):
        print(f"256x256 image file does not exist: {ids_256}")
        continue

    img_x = read_image(ids_256)     # x: 256x256 img
    if img_x is None:
        print(f"Skipping image at {ids_256}")
        continue

    img_x = image_preprocess(img_x, training_mode=False)

    x_train_chunk.append(np.array(img_x, np.float32) / 255.)
    y_train_chunk.append(np.array(img_x, np.float32) / 255.)  # Same image for both x and y for training

x_train_chunk = np.array(x_train_chunk)
y_train_chunk = np.array(y_train_chunk)

print(f"Processed {len(x_train_chunk)} training image pairs.")

# Prepare testing data
x_test_chunk = []
y_test_chunk = []
test_chunk_size = 10  # Adjust the chunk size as needed
ids_test_chunk = test_list[:test_chunk_size]

for i, ids in enumerate(ids_test_chunk):
    # Construct the path for the corresponding 128x128 image
    ids_128 = ids.replace(test_size, test_size)
    print(f"Processing: 128x128 image at {ids_128}")

    # Check if the 128x128 image file exists
    if not os.path.isfile(ids_128):
        print(f"128x128 image file does not exist: {ids_128}")
        continue

    img_x = read_image(ids_128)     # x: 128x128 img
    if img_x is None:
        print(f"Skipping image at {ids_128}")
        continue

    img_x = image_preprocess(img_x, training_mode=False)

    x_test_chunk.append(np.array(img_x, np.float32) / 255.)
    y_test_chunk.append(np.array(img_x, np.float32) / 255.)  # Same image for both x and y for testing

x_test_chunk = np.array(x_test_chunk)
y_test_chunk = np.array(y_test_chunk)

print(f"Processed {len(x_test_chunk)} testing image pairs.")



In [ ]:
x_chunk.shape

In [ ]:
show_pictures(4, x_chunk, y_chunk)
show_pictures(5, x_chunk, y_chunk)
show_pictures(9, x_chunk, y_chunk)